In [799]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import math

#reading in the data file
wellData = pd.read_csv("Data/well production.csv")
wellData.head()

,well name,average pressure (Pa),recovery factor,formation volume factor,oil 1,oil 2,oil 3,oil 4,oil 5,oil 6,...,water 3,water 4,water 5,water 6,water 7,water 8,water 9,water 10,water 11,water 12
0,Peak 6-217H,35352874,0.092554,1.6,862.0,824.0,759.0,728.0,661.0,601.0,...,180.0,211.0,277.0,338.0,397.0,445.0,547.0,562.0,639.0,668.0
1,Tarragon 4-119H,34882173,0.107706,1.6,228.0,249.0,214.0,210.0,161.0,163.0,...,22.0,27.0,75.0,74.0,59.0,90.0,124.0,119.0,126.0,157.0
2,Fennel 10-129H,36064538,0.079150,1.6,67.0,85.0,73.0,73.0,57.0,58.0,...,15.0,15.0,31.0,30.0,33.0,31.0,20.0,49.0,30.0,41.0
3,Federal 14-113H,35817881,0.103748,1.6,256.0,242.0,267.0,263.0,199.0,191.0,...,9.0,13.0,78.0,86.0,119.0,134.0,139.0,162.0,136.0,183.0
4,King 7-184H,38442406,0.084675,1.6,23.0,29.0,31.0,50.0,72.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0


In [800]:
# creating labels for the dataframe of relevant information
labels = ['stageCount', 'easting', 'northing'] + ['oil {}'.format(x) for x in range(1,13)] + ['total oil']
genData = pd.DataFrame(columns = labels)

In [801]:
# general dataframe with information about all wells
genData = pd.DataFrame(columns = ['name', 'length', 'stageCount', 'propWeight', 
                                  'pumpRate', 'oilProd', 'easting', 'northing'])

# for every well logged
for i in range(len(wellData.index)):
    name = wellData.iloc[i,0]
    # creating file name to be read
    fileName = f"Data/{name}.csv"
    tempDF = pd.read_csv(fileName)
    # storing csv of current well
    tempDF.columns = ['east', 'north', 'por', 'perm', 'poisson', 'young', 'waterSat', 
                      'oilSat', 'thick', 'propWeight', 'pumpRate']
    # finds total well length
    wellLen = tempDF.iloc[len(tempDF.index) - 1, 0] - tempDF.iloc[0, 0]
    # number of stages, used for averages
    stageCount = tempDF['propWeight'].count()
    # calculates average proppant weight per stage
    avgProp = tempDF['propWeight'].sum() / stageCount
    # calculates average pump rate for stage
    avgPumpRate = tempDF['pumpRate'].sum() / stageCount
    # calculates total yearly well production
    prod = wellData.iloc[i, 4 : 16].sum()
    #calculate locations of wells 
    easting = tempDF['east'].mean()
    northing = tempDF['north'].mean()
    #appends well-specific information into general dataframe for plotting
    genData = genData.append(pd.Series([name, wellLen, stageCount, avgProp, avgPumpRate, prod,
                                       easting, northing], 
                             index = genData.columns), ignore_index = True)

In [802]:
noName = genData[['length', 'stageCount', 'propWeight', 'pumpRate', 'easting', 'northing']]
y = genData['oilProd'].values
x_train, x_test, y_train, y_test = train_test_split(noName, y, test_size = .1)

n_estimators = [int(x) for x in np.linspace(start=100, stop=900, num=100)]
# the number of features to use at each split
max_features = ["auto", "sqrt"]
# max number of levels in each tree
max_depth = [int(x) for x in np.linspace(10, 220, num=11)]
max_depth.append(None)
# minimum samples needed to split a tree
min_samples_split = [2, 5, 10, 15, 20]
# minimum samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16]
# method for selecting samples
bootstrap = [True, False]


# create the grid
random_grid = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}

In [803]:
# creating the random forest regressor
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=19,
    n_jobs=-1,
)

# training the model
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   11.5s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [804]:
# function that compares the accuracy of model 
def compare(model, test_features, test_actual):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_actual)
    mape = 100 * np.mean(errors / test_actual)
    accuracy = 100 - mape
    print("Model Performance")
    print("Average Error: {:0.4f}".format(np.mean(errors)))
    print("Accuracy = {:0.2f}%.".format(accuracy))

    return accuracy

# checking the accuracy of a basic model
base_model = RandomForestRegressor(n_estimators=10)
base_model.fit(x_train, y_train)
base_accuracy = compare(base_model, x_train, y_train)

# checking the accuracy of the best model
best_random = rf_random.best_estimator_
random_accuracy = compare(best_random, x_train, y_train)

Model Performance
Average Error: 193.1656
Accuracy = 85.65%.
Model Performance
Average Error: 162.9755
Accuracy = 87.94%.


In [805]:
feat_labels = noName.columns.values  # get the feature labels
feature = list(
    zip(feat_labels, best_random.feature_importances_)
)  # make a list of the feature labels and the importance values
sorted(
    feature, key=lambda tup: tup[1], reverse=True
)  # sort from most to least important feature in predicting production

[('stageCount', 0.44416247636176087),
 ('northing', 0.20617705508482762),
 ('easting', 0.1829364069636029),
 ('propWeight', 0.06989272760578029),
 ('pumpRate', 0.05749974838339215),
 ('length', 0.03933158560063623)]

In [806]:
# values that aren't in model
# these will be used to compute averages for the final feature values
total_prop_weight = 0
total_pump_rate = 0
total_stage_count = 0
total_length = 0

In [769]:
# creating labels for the dataframe of relevant information
# note that proppant weight, pump rate, and length aren't included
# this is because we have deemed the effect these features have on the output as negligable 
labels = ['stageCount', 'easting', 'northing'] + ['oil {}'.format(x) for x in range(1,13)] + ['total oil']
genData = pd.DataFrame(columns = labels)

# adding information for all wells into dataframe
for i in range(len(wellData.index)):
    # gets name of well to access other data
    name = wellData.iloc[i,0]
    
    # reading csv of current well
    fileName = f"Data/{name}.csv"
    new_csv = pd.read_csv(fileName)
    
    # renaming columns for convenience
    new_csv.columns = ['east', 'north', 'por', 'perm', 'poisson', 'young', 'waterSat', 
                      'oilSat', 'thick', 'propWeight', 'pumpRate']
    
    # storing relevant information
    stageCount = new_csv['propWeight'].count()
    east = new_csv['east'].mean()
    north = new_csv['north'].mean()
    curr_list = [stageCount, east, north]
    
    oil_sum = 0
    for j in range(4,16):
        monthly_oil = 10 * wellData.iloc[i, j]
        curr_list.append(monthly_oil)
        oil_sum += monthly_oil
    curr_list.append(oil_sum)
        
    # adding well-specific information to general dataframe 
    genData = genData.append(pd.Series(curr_list, index = genData.columns), ignore_index = True)

In [770]:
genData = genData.drop(columns = ['total oil'])

In [773]:
for i in range(len(wellData.index)):
    name = wellData.iloc[i, 0]
    
    fileName = fileName = f"Data/{name}.csv"
    new_csv = pd.read_csv(fileName)
    
    new_csv.columns = ['east', 'north', 'por', 'perm', 'poisson', 'young', 'waterSat', 
                      'oilSat', 'thick', 'propWeight', 'pumpRate']
    
    curr_oil_sum = 0
    for j in range(4,16):
        monthly_oil = 10 * wellData.iloc[i, j]
        curr_oil_sum += monthly_oil
    #oil_sum.append(curr_oil_sum)    
    total_prop_weight += new_csv['propWeight'].sum()
    total_pump_rate += new_csv['pumpRate'].sum()
    total_stage_count += stageCount
    total_length += (new_csv.iloc[99,0] - new_csv.iloc[0,0])

In [774]:
# finding averages to determine 
average_prop_weight = total_prop_weight / total_stage_count
average_pump_rate = total_pump_rate / total_stage_count
average_length = total_length / 100


In [775]:
# features that will be used as input to train model
x = genData[['stageCount', 'easting', 'northing']]

# values that will be used as results to train model
y = genData.drop(columns = ['stageCount', 'easting', 'northing'])

# splitting data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .1)

# creating and training model
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(x_train, y_train)

# checking accuracy of model
rf.score(x_test, y_test)

//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.8788064572720073

In [776]:
# creating lists of possible feature values
easting_list = [int(x) for x in np.linspace(10000, 100000, num = 10)]
northing_list = [int(x) for x in np.linspace(5000, 100000, num = 20)]
stage_count_list = [int(x) for x in np.linspace(4, 100, num = 25)]

# list to store inputs matched with outputs
tuple_list = []

# iterating through every possible combination of feature values 
for i in range(len(easting_list)):
    for j in range(len(northing_list)):
        east = easting_list[i] - 5000
        north = northing_list[j] - 2500
        max_stage = pd.Series()
        max_val = pd.Series()
        for k in range(len(stage_count_list)):
            stage = stage_count_list[k] - 2
            
            # predicting oil production for current feature values
            curr_pred = rf.predict([[stage, east, north]])
            
            # store max feature values at each location
            if curr_pred.sum() > max_val.sum():
                max_val = curr_pred
                max_stage = stage
                
        # adds input and output as tuples
        tuple_list.append((stage, east, north, np.sum(max_val[0]), max_val[0]))

In [777]:
# sorts feature values by total oil output
def getKey(tuple):
    return tuple[3]

sorted_tuple_list = sorted(tuple_list, key = getKey, reverse = True)

In [778]:
#creating dataframe of 10 best wells, its features, and its production
gen_ind = ['frac stage', 'easting', 'northing']
oil_ind = ['oil {}'.format(x) for x in range(1, 13)]
gen_oil_ind = gen_ind + oil_ind
final_df = pd.DataFrame(columns = gen_oil_ind)

# storing only the 10 best predicted wells
newList = [sorted_tuple_list[i] for i in range(10)]

# reorganizing data to store into final dataframe
for i in range(len(newList)):
    cur_info = []
    for j in range(3):
        cur_info.append(newList[i][j])
    oil_data = newList[i][4]
    
    # adding data
    gen_series = pd.Series(cur_info, index = gen_ind)
    gen_series = gen_series.append(pd.Series(oil_data[:12], index = oil_ind))
    final_df = final_df.append(gen_series, ignore_index = True)

In [780]:
# funtion that calcualtes decline rate
def get_decline_rate(newList, i):
    oil_list = newList[i][4]
    sum = 0
    for j in range(1, len(oil_list)):
        sum += oil_list[j] / oil_list[j - 1]
    return sum / 11

# function that calculates reservoir lifespan
def get_lifespan(decline_rate, initial_rate, econ_limit):
    return (1 / decline_rate) * math.log(initial_rate / econ_limit)

eur_list = []

# calculting the EUR for each predicted well
for i in range(len(newList)):
    initial_rate = newList[i][4][0]
    decline_rate = get_decline_rate(newList, i)
    lifespan = get_lifespan(decline_rate, initial_rate, 60)
    total_prod = 0
    for j in range(int(life_span * 12) + 1):
        total_prod += initial_rate
        initial_rate *= decline_rate
    eur_list.append(total_prod)

# adding EURs to dataframe
final_df['EUR'] = eur_list

In [781]:
# general dataframe with information about all wells
genData = pd.DataFrame(columns = ['name', 'length', 'stageCount', 'propWeight', 
                                  'pumpRate', 'oilProd', 'easting', 'northing', 'ooip', 'rr'])

# for every well logged
for i in range(len(wellData.index)):
    name = wellData.iloc[i,0]
    
    # creating file name to be read
    fileName = f"Data/{name}.csv"
    tempDF = pd.read_csv(fileName)
    
    # storing csv of current well
    tempDF.columns = ['east', 'north', 'por', 'perm', 'poisson', 'young', 'waterSat', 
                      'oilSat', 'thick', 'propWeight', 'pumpRate']
    
    # finds total well length
    wellLen = tempDF.iloc[len(tempDF.index) - 1, 0] - tempDF.iloc[0, 0]
    
    # number of stages, used for averages
    stageCount = tempDF['propWeight'].count()
    
    # calculates average proppant weight per stage
    avgProp = tempDF['propWeight'].sum() / stageCount
    
    # calculates average pump rate for stage
    avgPumpRate = tempDF['pumpRate'].sum() / stageCount
    
    # calculates total yearly well production
    prod = wellData.iloc[i, 4 : 16].sum()
    
    #calculate locations of wells 
    easting = tempDF['east'].mean()
    northing = tempDF['north'].mean()
    
    # calculate reservoir area   
    volProppant = avgProp / 100 # density of sand is around 100 lb / ft^3
    totalVol = .2 * volProppant # porosity of sand
    height = tempDF['thick'].mean() # avg thickness of reservoir
    width = .0656 # 2 cm to feet
    b = totalVol / width / height / 2 # half frac length = vol / width / height / 2
    res_area = (wellLen * 2 * b + math.pi * (b ** 2) ) / 43650
    
    # calculate thickness of reservoir
    avg_thickness = tempDF['thick'].mean()
    
    # calculate avg porosity
    avg_porosity = tempDF['por'].mean()
    
    # calculate avg water saturation
    avg_waterSat = tempDF['waterSat'].mean()
    
    # calculate formation volume factor
    fvf = 1.6
    
    # calculate ooip
    ooip = 7758 * res_area * avg_thickness * avg_porosity * (1 - avg_waterSat) / fvf
    
    # calculate rr
    rr = ooip * wellData.iloc[i, 2]
    
    #appends well-specific information into general dataframe for plotting
    genData = genData.append(pd.Series([name, wellLen, stageCount, avgProp, avgPumpRate, prod,
                                       easting, northing, ooip, rr], 
                             index = genData.columns), ignore_index = True)

In [782]:
locData = genData[["easting", "northing"]]

In [783]:
# model outputs ooip
ooip_y = genData['ooip'].values
ooip_x_train, ooip_x_test, ooip_y_train, ooip_y_test = train_test_split(locData, ooip_y, 
                                                                        test_size = .1)

#model outputs RR
rr_y = genData['rr'].values
rr_x_train, rr_x_test, rr_y_train, rr_y_test = train_test_split(locData, rr_y, 
                                                                test_size = .1)

In [784]:
# creating models 
ooip_rf = RandomForestRegressor()
rr_rf = RandomForestRegressor()

# training models
ooip_rf.fit(ooip_x_train, ooip_y_train)
rr_rf.fit(rr_x_train, rr_y_train)


//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [785]:
# making lists for OOIP and RR to add to dataframe
ooip_list = []
rr_list = []

# predicting OOIP and RR for each predicted well's location
for i in range(len(final_df.index)):
    east = final_df.iloc[i, 1]
    north = final_df.iloc[i, 2]
    ooip_list.append(ooip_rf.predict([[east, north]])[0])
    rr_list.append(rr_rf.predict([[east, north]])[0])

# adding OOIP and RR predictions to dataframe
final_df['OOIP'] = ooip_list
final_df['RR'] = rr_list
final_df['length'] = [average_length] * 10

In [786]:
final_df = final_df.drop(columns = ['oil {}'.format(x) for x in range(1, 13)])
final_df.head()

,frac stage,easting,northing,EUR,OOIP,RR,length
0,98.0,75000.0,17500.0,53102.892014,1.523456e+06,152915.854675,7706.73
1,98.0,75000.0,22500.0,54185.827299,1.492918e+06,143559.728976,7706.73
2,98.0,75000.0,12500.0,52352.157029,1.759825e+06,215194.450138,7706.73
3,98.0,75000.0,27500.0,54252.939793,1.238899e+06,122108.519098,7706.73
4,98.0,75000.0,7500.0,52155.164610,1.833543e+06,195190.212508,7706.73


In [787]:
# adding proppant weight and pump rate
pump_rate_list = []

for i in range(len(final_df.index)):
    frac_count = final_df.iloc[i, 0]
    # prop_weight_list.append(frac_count * average_prop_weight)
    pump_rate_list.append(frac_count * average_pump_rate)
    
final_df['proppant weight (per stage)'] = [average_prop_weight] * 10
final_df['pump rate (total)'] = pump_rate_list

final_df.head()

,frac stage,easting,northing,EUR,OOIP,RR,length,proppant weight (per stage),pump rate (total)
0,98.0,75000.0,17500.0,53102.892014,1.523456e+06,152915.854675,7706.73,646889.314165,24059.718827
1,98.0,75000.0,22500.0,54185.827299,1.492918e+06,143559.728976,7706.73,646889.314165,24059.718827
2,98.0,75000.0,12500.0,52352.157029,1.759825e+06,215194.450138,7706.73,646889.314165,24059.718827
3,98.0,75000.0,27500.0,54252.939793,1.238899e+06,122108.519098,7706.73,646889.314165,24059.718827
4,98.0,75000.0,7500.0,52155.164610,1.833543e+06,195190.212508,7706.73,646889.314165,24059.718827


In [788]:
def printer(i, frac_stage, easting, northing, eur, ooip, rr, length, proppant, pump_rate):
    print('For well #{}, we propose a length of {} ft with an easting of {} and northing of {}.'
          .format(i + 1, length, easting, northing))
    print('This well would have an EUR of {} barrels, an OOIP of {} barrels, and an RR of {} barrels.'
         .format(eur, ooip, rr))
    print('It would have {} frac stages with each stage using {} lbs of proppant with a pump rate of {} cubic feet/min'
          .format(frac_stage, proppant, pump_rate))
    print("\n")

In [789]:
for i in range(len(final_df.index)):
    frac_stage = final_df.iloc[i, 0]
    easting = final_df.iloc[i, 1]
    northing = final_df.iloc[i, 2]
    eur = final_df.iloc[i, 3]
    ooip = final_df.iloc[i, 4]
    rr = final_df.iloc[i, 5]
    length = final_df.iloc[i, 6]
    proppant_weight = final_df.iloc[i, 7]
    pump_rate = final_df.iloc[i, 8]
    printer(i, frac_stage, easting, northing, eur, ooip, rr, length, proppant_weight, pump_rate)

For well #1, we propose a length of 7706.73 ft with an easting of 75000.0 and northing of 17500.0.
This well would have an EUR of 53102.89201354829 barrels, an OOIP of 1523455.7708624953 barrels, and an RR of 152915.85467518555 barrels.
It would have 98.0 frac stages with each stage using 646889.3141649393 lbs of proppant with a pump rate of 24059.718827170433 cubic feet/min


For well #2, we propose a length of 7706.73 ft with an easting of 75000.0 and northing of 22500.0.
This well would have an EUR of 54185.82729926603 barrels, an OOIP of 1492918.0985650234 barrels, and an RR of 143559.7289760074 barrels.
It would have 98.0 frac stages with each stage using 646889.3141649393 lbs of proppant with a pump rate of 24059.718827170433 cubic feet/min


For well #3, we propose a length of 7706.73 ft with an easting of 75000.0 and northing of 12500.0.
This well would have an EUR of 52352.157028930436 barrels, an OOIP of 1759824.5719719876 barrels, and an RR of 215194.4501383452 barrels.
It w